In [1]:
from transformers import AutoConfig, AutoTokenizer, AutoModelForMaskedLM

config = AutoConfig.from_pretrained('vinai/bertweet-base')
tokenizer = AutoTokenizer.from_pretrained('vinai/bertweet-base')
# model = AutoModelForMaskedLM.from_config(config)
model = AutoModelForMaskedLM.from_pretrained('vinai/bertweet-base')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [2]:
from glob import glob
from datasets import Dataset

import re
import pandas as pd

from parse import mask_data_loading

data_url = '../crawler/stock/data/**.json'
url = glob(data_url)[-1]
data, symbols = mask_data_loading(url, tokenizer, symbol_mask=False)

dataset = Dataset.from_pandas(data.loc[:, ['labels', 'sentense']])
dataset = dataset.remove_columns('__index_level_0__')
dataset = dataset.train_test_split(test_size=0.2)

train_dataset = dataset['train']
test_dataset = dataset['test']
# dataset = dataset.shuffle().select(range(50000))

special_tokens_dict = {'additional_special_tokens': list(symbols)}
tokenizer.add_special_tokens(special_tokens_dict)
model.resize_token_embeddings(len(tokenizer))

Embedding(65862, 768)

In [3]:
# tokenizer.save_pretrained('./symbol-vocab')

In [4]:
def encode(example):
    sentense = example['sentense']
    label = example['labels']

    result = tokenizer(sentense, padding=True, truncation=True)

    return result

context_length = 128
def tokenize(element):
    outputs = tokenizer(
        element["sentense"],
        padding="max_length",
        truncation=True,
        max_length=context_length,
        return_overflowing_tokens=True,
        return_length=True,
    )
    input_batch = []
    for length, input_ids in zip(outputs["length"], outputs["input_ids"]):
        if length <= context_length:
            input_batch.append(input_ids)
    return {"input_ids": input_batch}

encoded_train_dataset = train_dataset.map(tokenize, batched=True, remove_columns=train_dataset.column_names)
encoded_test_dataset = test_dataset.map(tokenize, batched=True, remove_columns=test_dataset.column_names)

100%|██████████| 28/28 [00:14<00:00,  1.95ba/s]


In [5]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.1)

samples = encoded_train_dataset[:2]
a = data_collator(samples['input_ids'])
print(tokenizer.decode(a['input_ids'][0]))

<s> $TSLA 2k on tsla today thanks bears. Started with <unk> 85 <unk> <unk> </s> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>


# Training

In [6]:
import numpy as np
from datasets import load_metric

metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [7]:
from transformers import TrainingArguments, Trainer, Seq2SeqTrainingArguments, Seq2SeqTrainer
# Seq2SeqTrainer
training_args = Seq2SeqTrainingArguments(
    output_dir="after-bert-random-trainer",
    per_device_train_batch_size=4,
    num_train_epochs=5,
    save_steps=50000
)


trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=encoded_train_dataset,
    eval_dataset=encoded_test_dataset,
    compute_metrics=compute_metrics,
)

trainer.train()

C:\Users\douli\anaconda3\envs\stock-nlp\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 109456
  Num Epochs = 5
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 136820
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: alan8365 (use `wandb login --relogin` to force relogin)


  0%|          | 501/136820 [01:10<7:18:10,  5.19it/s]

{'loss': 5.443, 'learning_rate': 4.9817278175705304e-05, 'epoch': 0.02}


  1%|          | 1001/136820 [02:18<7:16:32,  5.19it/s]

{'loss': 3.9724, 'learning_rate': 4.963455635141062e-05, 'epoch': 0.04}


...

{'loss': 2.9704, 'learning_rate': 1.3638356965355941e-05, 'epoch': 3.64}


 73%|███████▎  | 100000/136820 [3:33:03<2:11:19,  4.67it/s]Saving model checkpoint to after-bert-random-trainer\checkpoint-100000
Configuration saved in after-bert-random-trainer\checkpoint-100000\config.json


{'loss': 2.9557, 'learning_rate': 1.3455635141061248e-05, 'epoch': 3.65}


Model weights saved in after-bert-random-trainer\checkpoint-100000\pytorch_model.bin
 73%|███████▎  | 100358/136820 [3:34:02<1:23:19,  7.29it/s] 

KeyboardInterrupt: 

In [8]:
from transformers import pipeline

fill_mask = pipeline(
    "fill-mask",
    model=model,
    tokenizer=tokenizer,
    device=0
)

s = f'{tokenizer.mask_token} to the sky!!!'
fill_mask(s)

[{'score': 0.14990156888961792,
  'token': 1244,
  'token_str': 'B a c k',
  'sequence': 'Back to the sky!!!'},
 {'score': 0.041228100657463074,
  'token': 53632,
  'token_str': 's o a r i n g',
  'sequence': 'soaring to the sky!!!'},
 {'score': 0.038240138441324234,
  'token': 107,
  'token_str': 'b a c k',
  'sequence': 'back to the sky!!!'},
 {'score': 0.03203132748603821,
  'token': 22482,
  'token_str': 'r o c k e t s',
  'sequence': 'rockets to the sky!!!'},
 {'score': 0.03099006786942482,
  'token': 2121,
  'token_str': 's h o o t',
  'sequence': 'shoot to the sky!!!'}]

In [9]:
import torch
torch.cuda.empty_cache()
print(torch.cuda.memory_summary())

|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |    2099 MB |    2892 MB |  334254 GB |  334252 GB |
|       from large pool |    2095 MB |    2869 MB |  332165 GB |  332163 GB |
|       from small pool |       4 MB |      24 MB |    2088 GB |    2088 GB |
|---------------------------------------------------------------------------|
| Active memory         |    2099 MB |    2892 MB |  334254 GB |  334252 GB |
|       from large pool |    2095 MB |    2869 MB |  332165 GB |